Intro about predicting blueberry harvest.

Set up data

In [ ]:
from pathlib import Path
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import dtreeviz
from IPython.display import Image, display_svg, SVG

data_path = Path('./data')
df = pd.read_csv(data_path/'train.csv', low_memory=False)
df_test = pd.read_csv(data_path/'test.csv', low_memory=False)
df

EDA

In [ ]:
df.columns
df_test.columns
[{df[i].name:df[i].unique()} for i in df.columns]

Based on how few unique values they contain, the first five columns might warrant treatment as categorical variables

In [ ]:
df.clonesize = df.clonesize.astype('category')
df.honeybee = df.honeybee.astype('category')
df.bumbles = df.bumbles.astype('category')
df.andrena = df.andrena.astype('category')
df.osmia = df.osmia.astype('category')

The dependent variable we are predicting is yield, and procs are wrappers on Pandas that handle strings and missing data. This dataset does not contain strings bu the functionality are grouped together. `Categorify` is a `TabularProc` that replaces a column with a numeric categorical column. `FillMissing` is a `TabularProc` that replaces missing values with the median of the column, and creates a new Boolean column that is set to `True` for any row where the value was missing.

In [ ]:
dep_var = 'yield'
procs = [Categorify, FillMissing]

Based on the structure of the dataset, we will randomly split the data into train and validation sets

In [ ]:
df_test
df
rng = np.random.default_rng()
np.random.seed(11)
train_size = round(len(df) * .7)
train_idx = rng.integers(low=0, high=df.last_valid_index(), size=train_size)
splits = (list(train_idx), list(df.index[~train_idx]))

Tell TabularPandas which columns are continuous and categorical. Save the processed data for later use.

In [ ]:
cont,cat = cont_cat_split(df, 1, dep_var=dep_var)
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)
to.show(3)
save_pickle(data_path/'to.pkl',to)

Decision tree
First define x and y, the independent and dependent variables. Then create the decision tree.

In [ ]:
trn_xs,trn_y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y
m = DecisionTreeRegressor(max_leaf_nodes=4)
m.fit(xs, y);

This function visualizes the decision tree for the training x data that's been passed to the tabularpandas. The first node is before anything has been done. The value is the mean of the variable we're trying to predict, yield, and the sample is the length of the dataframe.

In [ ]:
import graphviz

def draw_tree(t, df, size=10, ratio=0.6, precision=2, **kwargs):
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True, rounded=True,
                      special_characters=True, rotate=False, precision=precision, **kwargs)
    return graphviz.Source(re.sub('Tree {', f'Tree {{ size={size}; ratio={ratio}', s))
draw_tree(m, trn_xs, size=10)

We can confirm by checking summary statistics on the training set

In [ ]:
df.iloc[train_idx].describe()

The next two nodes come from bisecting the dataset by values for fruitset above and below 0.5.

Alternative visualization using dtreeviz, showing the distribution of data along with the bisecting lines

In [ ]:
dtreeviz.model(m, X_train=xs, y_train=y, feature_names=df.columns, target_name=dep_var).view()

These previews were limited to just 4 nodes but now we will remove it

In [ ]:
m = DecisionTreeRegressor(max_leaf_nodes=4)
m.fit(xs, y);

The Kaggle competition that this data was taken from will evaluate models based on mean absolute error

{{< katex >}}
`\(( MAE = \frac{1}{n}\sum_{i=1}^n|x_1 - y_1| \))`{=markdown}

where each `\\( x_i \\)`{=markdown} represents the predicted target, `\\( y_i \\)`{=markdown}  represents the ground truth, and `\\( n \\)`{=markdown} is the number of rows in the test set.